In [3]:
#coding:gb18030
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
#启动spark
spark = SparkSession.builder.master("local").appName("test").getOrCreate()
#sc = spark.sparkContext

In [5]:
# 读取json文件格式
df = spark.read.json('people.json')
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
# 打印模式
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
# 选择多列
df.select(df.name, df.age + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [10]:
# 条件过滤
df.filter(df.age > 20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [11]:
# 分组聚合
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [12]:
#排序
df.sort(df.age.desc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [13]:
# 多列排序
df.sort(df.age.desc(), df.name.asc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [14]:
# 对列进行重命名
df.select(df.name.alias("username"), df.age).show()

+--------+----+
|username| age|
+--------+----+
| Michael|null|
|    Andy|  30|
|  Justin|  19|
+--------+----+



In [15]:
# 利用反射机制推断出rdd模式
from pyspark.sql.types import Row
def f(x):
    rel = {}
    rel['name']= x[0]
    rel['age'] = x[1]
    return rel
peopleDF = sc.textFile('people.txt').map(lambda line:line.split(',')).map(lambda x:Row(**f(x))).toDF()
peopleDF.createOrReplaceTempView("people")
personDF = spark.sql("select * from people")
personDF.rdd.map(lambda t: "Name:" + t[0] + "," + "Age:" + t[1]).foreach(print)

In [20]:
# 使用编程方式定义RDD
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

peopleRDD = sc.textFile("people.txt")
schemaString = "name age"
# 根据模式字符串生成模式
fields = list(map(lambda fieldName: StructField(fieldName, StringType(), nullable=True), schemaString.split(" ")))
schema = StructType(fields)
rowRDD = peopleRDD.map(lambda line:line.split(',')).map(lambda attributes: Row(attributes[0], attributes[1]))
peopleDF = spark.createDataFrame(rowRDD, schema)
peopleDF.createOrReplaceTempView("people")
results = spark.sql("select * from people")
results.rdd.map(lambda attributes: "name: " + attributes[0] + ", " +"age:" + attributes[1]).foreach(print)

In [31]:
# rdd保存成文件
peopleDF = spark.read.format("json").load("people.json")
# 暂时没找出原因 后续再查一下
# peopleDF.rdd.saveAsTextFile("file:///C:/Users/wd/Documents/GitHub/Datawhale_DA/newpeople1.txt")

In [ ]:
# SparkStreaming程序基本步骤
#1.通过创建输入DStream来定义输入源
#2.通过对DStream应用转换操作和输出操作来定义流计算。
#3.用streamingContext.start()来开始接收数据和处理流程。
#4.通过streamingContext.awaitTermination()方法来等待处理结束（手动结束或因为错误而结束）。
#5.可以通过streamingContext.stop()来手动结束流计算进程。

In [36]:
# 创建pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# 每隔一秒自动执行一次流计算
ssc = StreamingContext(sc, 1)

In [35]:
# 如果是编写独立的SparkStreaming程序 
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
#conf = SparkConf()
#conf.setAppName("TestDStream")
#表示在本地运行 且启动两个进程
#conf.setMaster("local[2]")
#sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 1)